# Schedule for one night with respect to objects priority.

## Cool things

* we use patterns for ra and dec so users can write it in different and comfortable ways for them

## Libraries

In [1]:
#setting up the libraries and packages to work with
import numpy as np #here we abbreviate "numpy" as "np" because we will use it with its commands like "np.command"
import pandas as pd
from astropy.time import Time
#import astroplan
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from pytz import timezone
from tabulate import tabulate
import re

import time

from astropy.utils.iers import conf
conf.auto_max_age = None #Maximum age (days) of predictive data before auto-downloading. Default is 30.

## Constant part

### Location

In [3]:
longitude_tshao = '76d55m45.00s'
latitude_tshao = '43d15m49.00s'
elevation_tshao = 2735 * u.m
location_tshao = EarthLocation.from_geodetic(longitude_tshao, latitude_tshao, elevation_tshao)

### Tech time

In [4]:
# Read-out time
readout_1bin = 20# * u.second
readout_2bin = 14# * u.second
readout_3bin = 10# * u.second

# Filter block turn time
fil_turn_near = 5# * u.second
fil_turn_far = 10# * u.second

# Telescope turning speed
slew_rate = .8 #deg/second 

### Functions

#### Observatory --- DONE

In [5]:
class Observatory:
    """
    date                   --- date-time at midnight [Time function, one by one, not list]
                               (if observations are going to be at night, for instance, from 1st to 2d of
                               March 2023 you have to give as input Time("2023-03-02 00:00:00")),
    observatory_name       --- short name of observatory [str],
    longitude, latitude    --- coorinates of observatory [str, "**d**m**.**s"], 
    elevation              --- altitude above sea level [astropy units - meters, by default 0*u.m]
    pressure               --- [astropy units - bar, by default 0*u.bar],
    temperature            --- [astropy units - degrees of C, by default 0*u.deg_C]
    """
    def __init__(self,date, observatory_name, longitude, latitude, elevation=0*u.m,pressure=0*u.bar,temperature=0*u.deg_C):
        
        self.date = date
        self.observatory_name = observatory_name
        self.longitude = longitude
        self.latitude = latitude
        self.elevation_tshao = elevation
        self.pressure = pressure
        self.temperature = temperature
        self.location = EarthLocation.from_geodetic(longitude, latitude, elevation)
        self.observer = Observer(name=self.observatory_name,
                                 location=self.location,
                                 pressure=self.pressure,
                                 temperature=self.temperature)
    
    def sunset_rise_time(self):
    
        """
        ___________
        Description
        ___________
        Calculate sunset and surise time (astronomical twilight). 
        Also, calculate observation start and end in SIDEREAL time 
        
        ______
        Output
        ______
        Returns sunset and rise time as [Time function], sidereal time
        at midnight, sunset and rise (astronomical twilight) moments [float],
        and time range from set to rise time as [list of Time functions].
        All times will be given for UT+0h.
        
        return sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        
        _____
        Input
        _____
        date     --- date-time at midnight [Time function, one by one, not list]
                     (if observations are going to be at night, for instance, from 1st to 2d of
                     March 2023 you have to give as input Time("2023-03-02 00:00:00"))
        observer --- observatory [Astroplan Observer function,one by one, not list] 
        
        >>> sunset_rise_time(Time("2023-03-02 00:00:00"),Observer(location=EarthLocation.from_geodetic('76d58m17.00s', '43d03m29.00s', 2735 * u.m)))
        <Time object: scale='utc' format='jd' value=2460005.093717037>,
        <Time object: scale='utc' format='jd' value=2460005.495337549>,
        <Longitude 5.98912107 hourangle>,
        <Longitude 15.76660942 hourangle>,
        <Longitude 15.65440423 hourangle>,
        [<Time object: scale='utc' format='jd' value=2460005.093717037>,
         <Time object: scale='utc' format='jd' value=2460005.495337549>]
        """
        date = self.date
        observer = self.observer
        
        #Sunset and sunrise local sidereal time (same with strart and end of observation)
        sunset_tonight = observer.sun_set_time(date,  horizon = -18*u.degree, which='nearest') #in UT 
        sunrise_tonight = observer.sun_rise_time(date,  horizon = -18*u.degree, which='nearest')  #in UT
        
        #Sidereal time at sunset, sunrise and midnight moments
        sid_start =  observer.local_sidereal_time(sunset_tonight).value  #in UT
        sid_end = observer.local_sidereal_time(sunrise_tonight).value #in UT
        sid_mid = observer.local_sidereal_time(date).value #in UT
        
        #time range between sunset and sunrise moments
        time_range = [sunset_tonight, sunrise_tonight]
    
        self.sunset = sunset_tonight
        self.sunrise = sunrise_tonight
        self.sir_start = sid_start
        self.sid_mid = sid_mid
        self.sid_end = sid_end
        self.time_range = time_range
        
        #print("self.sunset",self.sunset.iso)
        #print("self.sunrise",self.sunrise.iso)
        
        
        return sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
    
    
    def parking_coordinates(self,date=None):

        """
        ___________
        Description
        ___________
        As we know, telescopes are parked usually on the South and paralell to the graund.
        So we have horizontal coordinates: azimut A = 270 deg and height  h = 0 deg.
        We need only two variable: lattitude of observatory (phi) and date of observations.
        
        ______
        Output
        ______
        Returns equatorial coordinates (RA,DEC) of telescope parking point.
        
        return parking_alpha_deg, parking_delta_deg, parking_alpha_hms, parking_dec_dms
        _____
        Input
        _____
        date     --- date-time at sunset/sunrise moment [Time function, one by one, not list]
        observer --- observatory [Astroplan Observer function,one by one, not list] 
        
        """
        if date:
            date=date
        else:
            date = self.date
            
        observer = self.observer
        
        A = 180*u.degree #because telescope should be parked "looking" to the south
        z = 0*u.degree # because telescope should be parked so that the pipe is parallel to the floor
    
        phi = observer.location.lat
        dec_rad = np.arcsin(np.sin(phi) * np.cos(z) - np.cos(phi) * np.sin(z) * np.cos(A))#* 360 / (2 * 3.14159265)
        parking_dec_deg = dec_rad.to(u.degree)

        JD2 = date.jd #date in JD to future calculations
        T = (JD2 - 2433282.5) / 36524.2    #date in fraction of 100 years
        S0 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T) + (((0.0929 / 60) / 60) * T**2) #sidereal time in Greenwich in UT 
        S0 = S0 % 24 #to account for only 24 hours in a day
        s = S0 + observer.location.lon.value #UT sidereal time in local area 
        s = s%24 #to account for only 24 hours in a day
        sid_h = s - 6 #local sidereal time in local area
        parking_alpha_deg = s * 360 / 24
        
        a_h = int(parking_alpha_deg * 24 / 360)
        a_m = int((parking_alpha_deg * 24 / 360 - a_h) * 60)
        a_s = round(((parking_alpha_deg * 24 / 360 - a_h) * 60 - a_m)*60,2)
        parking_alpha_hms = f'{a_h} {a_m} {a_s}'
    
        d_d = int(parking_dec_deg.value)
        d_m = int((parking_dec_deg.value - d_d) * 60)
        d_s = round(((parking_dec_deg.value - d_d) * 60 - d_m) * 60,2)
        parking_dec_dms = f'{d_d} {d_m} {d_s}'
        
        self.parking_alpha_deg = parking_alpha_deg 
        self.parking_dec_deg = parking_dec_deg 
        self.parking_alpha_hms = parking_alpha_hms 
        self.parking_dec_dms = parking_dec_dms
        
        return parking_alpha_deg, parking_dec_deg, parking_alpha_hms, parking_dec_dms

#### Telescope --- DONE

In [42]:
class Telescope:
    
    def __init__(self,readout_1bin=0,readout_2bin=0,readout_3bin=0,fil_turn_near=0,fil_turn_far=0,slew_rate=0):
       
    
        self.readout_1bin = readout_1bin
        self.readout_2bin = readout_2bin
        self.readout_3bin = readout_3bin
        self.fil_turn_near = fil_turn_near
        self.fil_turn_far = fil_turn_far
        self.slew_rate = slew_rate

#### Object --- DONE

In [34]:
class Object():
    """
    date          --- date-time at midnight [Time function, one by one, not list]
                      (if observations are going to be at night, for instance, 
                      from 1st to 2d of March 2023 you have to give as input 
                      Time("2023-03-02 00:00:00")),
    ra            --- right ascension of the first object 
                      [float or astropy units in DEGREES, one by one, not list],
    dec           --- declination of the first object 
                      [float or astropy units in DEGREES, one by one, not list],
    B,V,R         --- exposure time in respect filter 
                      [float or int, seconds, one by one, not list],
    bining        --- neccesary binning for object observations 
                      [int, one by one, not list],
    exp_num       --- necessary number of expositions in each filter
                          [int, one by one, not list]
    Telescope     --- Telescope object with some parameters
    Observatory   --- Observatory object with some parameters
    """
    
    def __init__(self,ra,dec, B,V,R,bining,exp_num,Telescope,Observatory):
        self.date = Observatory.date
        self.ra = ra
        self.dec = dec
        self.B = B
        self.V = V
        self.R = R
        self.bining = bining
        self.exp_num = exp_num
        self.Telescope = Telescope
        self.Observatory = Observatory
    
    def dec_to_deg(self,dec=None):
        """
        returns declanation as float in degrees.
    
        >>> dec_to_deg("29.06")
        29.06
        >>> dec_to_deg("-23.30")
        -23.3
        >>> "{:.5f}".format(dec_to_deg("50 41 45"))
        '50.69583'
        >>> "{:.5f}".format(dec_to_deg("-01 28 02"))
        '-1.46722'
        >>> "{:.5f}".format(dec_to_deg("-01 28"))
        '-1.46667'
        >>> "{:.5f}".format(dec_to_deg("-01d28m00s"))
        '-1.46667'
        >>> "{:.5f}".format(dec_to_deg("50d41m45"))
        '50.69583'
        """
        
        
        DEC_FORMATS = [re.compile(pat) for pat in [
          r"(?P<sign>-?)(?P<degrees>\d+\.?\d*)$",
          r"(?P<sign>-?)(?P<degrees>\d+) (?P<minutes>\d+)(?: (?P<seconds>\d+(?:.\d+)?))?$",
          r"(?P<sign>-?)(?P<degrees>\d+)d(?P<minutes>\d+(?:\.\d+)?)m(?:(?P<seconds>\d+)s?)?$",
          r"(?P<sign>-?)((?P<degrees>\d+):(?P<minutes>\d+):(?P<seconds>\d+(?:.\d+)?))$",
        ]]
        if dec:
            if str(type(dec)) == "<class 'str'>":
                raw_dec = str(dec).strip().replace("+","")
            else:
                return dec
        else:
            if str(type(self.dec)) == "<class 'str'>":
                raw_dec = self.dec.strip().replace("+","")
            else:
                return self.dec
            
        for pattern in DEC_FORMATS:
            mat = re.match(pattern, raw_dec)
            if mat:
                break
        else:
            raise ValueError(f"Not a valid Dec {raw_dec}")
    
        parts = mat.groupdict()
        dec_deg = (float(parts["degrees"])
            + float(parts.get("minutes", 0))/60.
            + float(parts.get("seconds", 0) or 0)/3600.)
    
        if parts["sign"]=="-":
            self.dec_deg = -dec_deg
            return -dec_deg
        else:
            self.dec_deg = dec_deg
            return dec_deg
    
    def ra_to_deg(self,ra=None):
        """
        returns declanation as float in degrees.
        
        >>> "{:.5f}".format(ra_to_deg("05 32 49"))
        '83.20417'
        >>> ra_to_deg("05h33m")
        83.25
        >>> "{:.4f}".format(ra_to_deg("02h41m45s"))
        '40.4375'
        >>> ra_to_deg("01 28")
        22.0
        """
        RA_FORMATS = [re.compile(pat) for pat in [
          r"(?P<hours>\d+) (?P<minutes>\d+)(?: (?P<seconds>\d+(?:.\d+)?))?$",
          r"(?P<hours>\d+)h(?P<minutes>\d+(?:\.\d+)?)m(?:(?P<seconds>\d+)s)?$",
          r"(?P<hours>\d+)h(?P<minutes>\d+(?:m?\.\d+)?)(?:(?P<seconds>\d+)s)?$",
          r"(?P<hours>\d+):(?P<minutes>\d+):(?P<seconds>\d+(?:.\d+)?)$",
        ]]
        if ra:
            if str(type(ra)) == "<class 'str'>":
                raw_ra = str(ra).strip()
            else:
                return ra
            
        else:
            if str(type(ra)) == "<class 'str'>":
                raw_ra = self.ra.strip()
            else:
                return self.ra
            
        for pattern in RA_FORMATS:
            mat = re.match(pattern, raw_ra)
            if mat:
                break
        else:
            raise ValueError(f"Not a valid RA {raw_ra}")
    
        parts = mat.groupdict()
        hours = (float(parts["hours"])
            + float(parts["minutes"].replace("m",""))/60.
            + float(parts["seconds"] or 0)/3600.)
        self.ra_deg = hours/24*360
        
        return hours/24*360
    
    
    def time_calculation(self):
    
        """
        ___________
        Description
        ___________
        To make schedule we have to know when the object will culminate,
        how much time we should observe it, time of start and finish of
        observations and will be ready to go to the next object or home...
        
        ______
        Output
        ______
        Returns start, end, culmination [Time function] and whole 
        observation time [astropy units - seconds].
        
        return start, end, culm, time
        _____
        Input
        _____
        date          --- date-time at midnight [Time function, one by one, not list]
                          (if observations are going to be at night, for instance, 
                          from 1st to 2d of March 2023 you have to give as input 
                          Time("2023-03-02 00:00:00")),
        ra            --- right ascension of the first object 
                          [float or astropy units in DEGREES, one by one, not list],
        dec           --- declination of the first object 
                          [float or astropy units in DEGREES, one by one, not list],
        B,V,R         --- exposure time in respect filter 
                          [float or int, seconds, one by one, not list],
        bining        --- neccesary binning for object observations 
                          [int, one by one, not list],
        exp_num       --- necessary number of expositions in each filter
                          [int, one by one, not list]
        sid_mid       --- sidereal time at midnight on the date of obeservations
                          [float],
        fil_turn_near --- turning the filter wheel; switching to the next closest filter
                          [float/int or astropy units - seconds],
        readout_1bin  --- signal readout time in 1st binning
                          [float/int or astropy units - seconds],
        readout_2bin  --- signal readout time in 2d binning
                          [float/int or astropy units - seconds],
        readout_3bin  --- signal readout time in 3d binning
                          [float/int or astropy units - seconds].
        """
        #FIND CULMINATION TIME IN UT
        #RA is culmination SIDEREAL timew
        ra_deg = Object.ra_to_deg(self,self.ra)
        
        ra_h = ra_deg*360/24
        self.Observatory.sunset_rise_time()
        delta = self.Observatory.sid_mid - ra_h
            
        if abs(delta) < 12:
            culm = self.date - delta*u.second - 6*u.hour
        elif abs(delta) > 12:
            delta = (24 - abs(delta))
            culm = self.date - delta*u.second - 6*u.hour
        #print("Object time_calculation",self.date)
        
        #NECESSARY DURATION OF OBSERVATIONS
        if self.bining == 1:
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second + (self.Telescope.readout_1bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        elif self.bining == 2: 
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second  + (self.Telescope.readout_2bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        elif self.bining == 3:
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second  + (self.Telescope.readout_3bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        
        delta_t = time / 2
        
        #start and end time of observations in respect to object culmination time
        start = culm - delta_t
        end = culm + delta_t
        
        self.start_obj_obs = start
        self.end_obj_obs = end
        self.culm_obj_obs = culm
        self.whole_time_obj_obs = time
        
        return start, end, culm, time
    
    
    def calc_obj_rise_set_time(self):
        
        """
        Returns object rise and set time in UT for Observatory.
        """
        cos_t = -np.tan(self.Observatory.location.lat)*np.tan(Object.dec_to_deg(self,self.dec)*u.degree)
        td_1 = np.arccos(cos_t)*u.rad.to(u.degree)
        td_2 = -np.arccos(cos_t)*u.rad.to(u.degree)
        
        td_1= td_1.value
        td_2= td_2.value
        
        if td_1 < 0 or td_1 > 360:
            td_1 = td_1%360
        if td_2 < 0 or td_2 > 360:
            td_2 = td_2%360
            
        th_1 = td_1*24/360 
        th_2 = td_2*24/360
            
        if th_2 > 12 and th_2 < 24: #rise point
            hour_angle_hour_rise = th_1
            hour_angle_hour_set = th_2
        else: #set point
            hour_angle_hour_rise = th_2
            hour_angle_hour_set = th_1

        sid_time_rise = (hour_angle_hour_rise + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        sid_time_set = (hour_angle_hour_set + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        
        time_rise = Object.convert_sid2ut(self,sid_time_rise)
        time_set = Object.convert_sid2ut(self,sid_time_set)
            
        visible_time = abs((time_set - time_rise).value) #seconds
        
        self.time_rise = time_rise 
        self.time_set = time_set 
        #print("time_rise, time_set, visible_time ",(time_rise+6*u.hour).iso.split(" ")[1], (time_set+6*u.hour).iso.split(" ")[1], round(visible_time*24,2))
        return time_rise, time_set, visible_time 
        

    def calc_obj_rise_set_15deg_time(self):
        
        """
        Returns object rise and set time in UT for Observatory
        in respect to 15 degree altitude, i.e. when altitude 
        of object became more than 15deg
        """
        cos_t = (np.cos((90-15)*u.degree) - np.sin(self.Observatory.location.lat)*np.sin(Object.dec_to_deg(self,self.dec)*u.degree)) / np.cos(self.Observatory.location.lat)*np.cos(Object.dec_to_deg(self,self.dec)*u.degree)

        td_1 = np.arccos(cos_t)*u.rad.to(u.degree)
        td_2 = -np.arccos(cos_t)*u.rad.to(u.degree)
        
        td_1= td_1.value
        td_2= td_2.value
        
        if td_1 < 0 or td_1 > 360:
            td_1 = td_1%360
        if td_2 < 0 or td_2 > 360:
            td_2 = td_2%360
            
        th_1 = td_1*24/360 
        th_2 = td_2*24/360
            
        if th_2 > 12 and th_2 < 24: #rise point
            hour_angle_hour_rise = th_1
            hour_angle_hour_set = th_2
        else: #set point
            hour_angle_hour_rise = th_2
            hour_angle_hour_set = th_1

        sid_time_rise = (hour_angle_hour_rise + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        sid_time_set = (hour_angle_hour_set + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        
        time_rise = Object.convert_sid2ut(self,sid_time_rise)
        time_set = Object.convert_sid2ut(self,sid_time_set)
            
        visible_time = abs((time_set - time_rise).value) #seconds
        
        self.time_rise = time_rise 
        self.time_set = time_set 
        #print("time_rise, time_set, visible_time ",(time_rise+6*u.hour).iso.split(" ")[1], (time_set+6*u.hour).iso.split(" ")[1], round(visible_time*24,2))
        return visible_time 

    def convert_sid2ut(self,sid_time):
        
        """
        Returns date-time as Time function.
        It uses sidereal time at neccessary date
        and then iteratively find exact time 
        by calculating sidereal time at each moment and
        compair it with know sidereal time.
        
        cos(z)=sin(lat)sin(DEC)+cos(lat)cos(DEC)cos(t)
        z = 90-15
        """  
        
        date = self.Observatory.date - 6*u.hour#time function
        lst_mid = date.sidereal_time("mean",self.Observatory.location.lon).value #float, UT+0h
        longitude = self.Observatory.location.lon
        date_final = None
        done = False
        hours = -12.5
        
        while done == False:
            hours += 1
            if hours > 11.5:
                print("Not found time")
                done = True
            else:
                date_new = date + hours*u.hour
                st = date_new.sidereal_time("mean",longitude).value #- longitude.value#GST
                if int(st)-int(sid_time) == 0: #we get an hour
                    #print("we get an hour")
                    for minutes in range(-60,61):
                        date_minutes = date_new + minutes*u.minute
                        st = date_minutes.sidereal_time("mean",longitude).value#GST
                        if abs(st - sid_time) <= 1/60: #we get a minute, i.e. accuracy is about 1 minute
                            #date_minutes = date_minutes 
                            date_final = date_minutes #+ 6*u.hour
                            #print("date_final ",date_final)
                            done = True
                            break
                            
                            #for seconds in range(-30,31):
                            #    date_seconds = date_minutes + seconds*u.second
                            #    st = date_seconds.sidereal_time("mean",longitude).value
                            #    if abs(st - sid_time) < 1/3600: #we get a second, i.e. accuracy is about 1 second
                            #        #date_minutes = date_minutes #+ 6*u.hour
                            #        date_final = date_seconds
                            #        done = True
                            #        break
                        else:
                            done = False
                else:
                    done = False
        #print(date_final)            
        return date_final 
    
    def decide_ascending_star(self):
        """
        Check if object is visible in exact location.
        Visibility means altitude in upper culmination point
        more than 0 deg.
        """
        
        dec_deg = Object.dec_to_deg(self)
        
        if dec_deg > 90 - self.Observatory.location.lat.value:
            
            return "Circumpolar"
        elif dec_deg < - (90  - self.Observatory.location.lat.value):
            return "Not ascending"
        else:
            return "Ascending"
        
    def get_culm_up_altitude(self):
        """
        Check if object is visible in exact location.
        Visibility means altitude in upper culmination point
        more than 0 deg.
        """

        dec_deg = Object.dec_to_deg(self)
        latitude = self.Observatory.location.lat.value
        
        altitude = 90 - latitude + dec_deg
        
        if altitude > 15: 
            return "Visible"
        else:
            return "Not ascending"
        
    def get_culm_low_altitude(self):
        """
        Check if object is visible in exact location.
        Visibility means altitude in upper culmination point
        more than 0 deg.
        """
        
        
        dec_deg = Object.dec_to_deg(self)
        latitude = self.Observatory.location.lat.value
        
        altitude = latitude + dec_deg - 90
        
        if altitude < 15: #everything more than 180-latitude.value-15 potentially 
            return "Ascending"
        else:
            return "Circumpolar"
        
    #def decide_ascending_15deg_star(self):
    #    """
    #    Check if object is observable in exact location.
    #    Observability means altitude in upper culmination point
    #    more than 15 deg, because if it is lower
    #    there is too high airmass.
    #    
    #    In upper culmination point A = 0.
    #    
    #    -sin(z)cos(A)=sin(DEC)cos(lat)-cos(DEC)cos(t)sin(lat)
    #    """
    #    dec_deg = Object.dec_to_deg(self)*u.degree
    #    latitude = self.Observatory.location.lat
    #    print()
    #    cos_h = np.cos(dec_deg)*np.sin(latitude) - np.sin(dec_deg)*np.cos(latitude)
    #    altitude = (np.arccos(cos_h)).to(u.degree)
    #    
    #    print(altitude)
    #    
    #    if altitude.value > 15: #everything more than 180-latitude.value-15 potentially 
    #        return "Ascending"
    #    else:
    #        return "Not ascending"
    #    
    #def decide_circumpolar_ascending(self):
    #    """
    #    Check if object altitude decrease under 15 degree.
    #    Under 15 degree there is too high airmass and 
    #    this object should be considered as ascending object.
    #    
    #    In lower culmination point A = 180.
    #    
    #    -sin(z)cos(A)=sin(DEC)cos(lat)-cos(DEC)cos(t)sin(lat)
    #    """
    #    dec_deg = Object.dec_to_deg(self)*u.degree
    #    latitude = self.Observatory.location.lat
    #    
    #    cos_h = np.sin(dec_deg)*np.cos(latitude) - np.cos(dec_deg)*np.sin(latitude)
    #    
    #    altitude = (np.arccos(cos_h)).to(u.degree)
    #    print(altitude)
    #    
    #    if altitude.value < 15 or altitude.value > 90:
    #        return "Ascending"
    #    else:
    #        return "Circumpolar"
        

In [ ]:
(23+56/60+4/3600)/2

In [ ]:
0.94*60

#### Table with ready schedule

In [83]:
class ScheduleTable():
    
    def __init__(self,date,table,Observatory,Telescope):
        self.date = date
        self.table = table
        #self.Object = Object
        self.Observatory = Observatory
        self.Telescope = Telescope
        
        self.table["N"]=self.table["exp_num"]*self.table["M"]
        

    def sort_target_visib(self,ra,dec,B,V,R,binning,exp_num):    
    
        """
        ___________
        Description
        ___________
        Check if RA of an object is in interval [sid_start,sid_end]. 
        In other words, could we observe an object culmination at night.
        Moreovere, we can observe object 2 hour before and after culmination, 
        so it is necessary to take it into account.
        
        ______
        Output
        ______    
        Returns object name [str] and altitude [float] if the object is observable
        and None if it is not observable.
        
        return obj_name, altitude
        _____
        Input
        _____    
        obj_name,ra,dec,phi,sid_start,sid_end
        
        obj_name  --- name of object [str, one by one, not list],
        ra        --- right ascension of object [float in degrees, one by one, not list],
        dec       --- declenation of object [float in degrees, one by one, not list],
        phi       --- latitude of observatory [float in degrees, one by one, not list],
        sid_start --- declenation of object [float, one by one, not list],
        sid_end   --- declenation of object [float, one by one, not list].
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time() #sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sid_start = observatory_time[2]
        sid_end = observatory_time[4]
        #target = Object(date, self.table["RA"][index], self.table["DEC"][index], self.table["B"][index], self.table["V"][index], self.table["R"][index], self.table["bining"][index], self.table["exp_num"][index], self.Telescope, self.Observatory)
        target = Object(ra,dec,B,V,R,binning,exp_num, self.Telescope, observatory)
        
        altitude = 90*u.degree + observatory.observer.location.lat.value*u.degree - target.dec_to_deg()*u.degree #formulae for obj on the north from zenith
        if altitude > 90*u.degree:
            altitude = 180*u.degree - altitude #formulae for obj on the south from zenith
            
        if altitude > 15*u.degree and altitude < 80*u.degree: #if < 18, then airmass > 3   !!!!!!!!!!!!!!!!18!!!!!!!!!!!!!!!!!
            
            ra_h = Object.ra_to_deg(self,target.ra) * 24 / 360
            if sid_start < sid_end:
                if (ra_h > (sid_start-2) and ra_h < (sid_end+2)): #we are able to observe target in range culm +- 2h
                    return round(altitude.value,3)
                else:
                    return None
            else:
                if (ra_h < (sid_end + 2) or ra_h > (sid_start - 2)): #we are able to observe target in range culm +- 2h
                    return round(altitude.value,3)
                else:
                    return None
        else:
            return None
                
                
    def calc_slew_time(self,ra_1,ra_2,dec_1,dec_2,slew_rate):
    
        """
        ___________
        Description
        ___________
        To make schedule we have to know how much time telescope will spend time
        to move from one object to another.
        
        ______
        Output
        ______
        Returns telescope slew time [float, in astropy units - seconds].
        
        return slew_time
        _____
        Input
        _____
        ra_1      --- right ascension of the first object 
                      [float or astropy units in degrees, one by one, not list],
        dec_1     --- declenation of the first object 
                      [float or astropy units in degrees, one by one, not list],
        ra_2      --- right ascension of the second object 
                      [float or astropy units in degrees, one by one, not list],
        dec_2     --- declenation of the second object 
                      [float or astropy units in degrees, one by one, not list],
        slew_rate --- rotate rate of telescope [float! 
                      u.deg/u.second, one by one, not list].
        """

        
        dist = ScheduleTable.calc_distance(self,ra_1,ra_2,dec_1,dec_2)
        
        slew_time = (dist / slew_rate) * u.second
        
        return slew_time
    
    def calc_distance(self,ra_1,ra_2,dec_1,dec_2):
        """
        _____
        Input
        _____
        ra_1      --- right ascension of the first object 
                      [float or astropy units in degrees, one by one, not list],
        dec_1     --- declenation of the first object 
                      [float or astropy units in degrees, one by one, not list],
        ra_2      --- right ascension of the second object 
                      [float or astropy units in degrees, one by one, not list],
        dec_2     --- declenation of the second object 
                      [float or astropy units in degrees, one by one, not list],
        """
        
        #Check if the input values are not in astropy units.
        
        if type(ra_1) != "<class 'astropy.units.quantity.Quantity'>":
            ra_1 = Object.ra_to_deg(self,ra_1)*u.degree
        if type(dec_1) != "<class 'astropy.units.quantity.Quantity'>":
            dec_1 = Object.dec_to_deg(self,dec_1)*u.degree
        if type(ra_2) != "<class 'astropy.units.quantity.Quantity'>":
            ra_2 =Object.ra_to_deg(self,ra_2)*u.degree
        if type(dec_2) != "<class 'astropy.units.quantity.Quantity'>":
            dec_2 = Object.dec_to_deg(self,dec_2)*u.degree
        
        distance = (np.arccos(np.sin(dec_1) * np.sin(dec_2) + np.cos(dec_1) * np.cos(dec_2) * np.cos(ra_1 - ra_2))).to(u.degree)
        
        return distance.value
    
    def change_priority_visib(self):
        
        table = self.table
        
        table["priority_vis"] = np.nan
        table["visible_time"] = np.nan
        table["visibility"] = np.nan
        
        for ind in range(len(table)):
            print("______________")
            print(table["name"][ind])
            target = Object(table["ra"][ind],table["dec"][ind],table["B"][ind],table["V"][ind],table["R"][ind],table["binning"][ind],table["exp_num"][ind], self.Telescope, self.Observatory)
            table["visible_time"][ind] = target.calc_obj_rise_set_15deg_time()
            if target.get_culm_up_altitude() == "Not ascending":
                table["visibility"][ind] = "Not ascending"
            else:
                table["visibility"][ind] = target.get_culm_low_altitude()
            
        table_ascend = table[table["visibility"]!="Not ascending"].reset_index(drop=True)
        
        maximum = table_ascend[table_ascend["visibility"]=="Ascending"]["visible_time"].max()
        
        for ind in range(len(table_ascend)):
            #target = Object(table["ra"][ind],table["dec"][ind],table["B"][ind],table["V"][ind],table["R"][ind],table["binning"][ind],table["exp_num"][ind], self.Telescope, self.Observatory)
            if table_ascend["visibility"][ind] == "Circumpolar":
                table_ascend["priority_vis"]=table_ascend["priority"]
            else:
                table_ascend["priority_vis"] = table_ascend["priority"]/(table_ascend["visible_time"][ind]/maximum)
        
        table_ascend_sorted = table_ascend.sort_values("ra")
        
        return table_ascend_sorted
        

    def calc_time_table(self,table):
        """
        name,ra,dec,B,V,R,bin,exp_num,start,end,altitude,culm.
        """
        table["altitude"] = np.nan        
        
        for index in range(len(table)):
            #check observable or not
            obj_alt = ScheduleTable.sort_target_visib(self,table["ra"][index],table["dec"][index],table["B"][index],table["V"][index],table["R"][index],table["binning"][index],table["exp_num"][index]) 
            if obj_alt == None: #if object is not observable we write Nan altitude to drop the row in the next step
                table["altitude"].iloc[index] = np.nan
            else:#if observable we add altitude data
                table["altitude"].iloc[index] = obj_alt
                continue
                
        #create new table with observable objects only    
        table_timing = table[table["altitude"].notna()].reset_index(drop=True)
        table_timing["start"] = None
        table_timing["end"] = None
        table_timing["culmination"] = None
        table_timing["time_obs"] = None
        #now we add new columns with object observation start and end time and object culmination time 
        for index in range(len(table_timing)):
            target = Object(table_timing["ra"][index],table_timing["dec"][index],table_timing["B"][index],table_timing["V"][index],table_timing["R"][index],table_timing["binning"][index],table_timing["exp_num"][index],self.Telescope,self.Observatory)
            table_timing["start"][index],table_timing["end"][index],table_timing["culmination"][index],table_timing["time_obs"][index] = target.time_calculation()
        for ind in range(1,len(table_timing)):
            #print("table start",type(table_timing['start'][ind]),table_timing['start'][ind])
            #print("slew time",type(ScheduleTable.calc_slew_time(self,table_timing['ra'][ind-1],table_timing['ra'][ind],table_timing['dec'][ind-1],table_timing['dec'][ind],self.Telescope.slew_rate)),ScheduleTable.calc_slew_time(self,table_timing['ra'][ind-1],table_timing['ra'][ind],table_timing['dec'][ind-1],table_timing['dec'][ind],self.Telescope.slew_rate))
            table_timing['start'][ind] = table_timing['start'][ind] - ScheduleTable.calc_slew_time(self,table_timing['ra'][ind-1],table_timing['ra'][ind],table_timing['dec'][ind-1],table_timing['dec'][ind],self.Telescope.slew_rate)
        
        #self.table_timing = table_timing
        #table_timing["ra_deg"] = None
        #for index in range(len(table_timing)):
        #    table_timing["ra_deg"][index]=Object.ra_to_deg(self,table_timing["ra"][index])
            
        table_timing=table_timing.sort_values(by="ra")
        #merged_Frame = pd.merge(table, table_timing[["name","start","end","culmination","altitude"]], on = "name", how='outer')
        
        return table_timing#merged_Frame

    def check_overlap(self,table=None):
        """
        Table with objects which WILL BE observed.
        Check if there overlap in time of obj observation.
        Sorting by priority.
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time() #sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunset_tonight = observatory_time[0]
        sunrise_tonight = observatory_time[1]
        #print(table)
        try:
            if table==None:
                table = self.table
        except ValueError:
            pass
        prior_table = ScheduleTable.calc_time_table(self,table)
        #prior_table = full_table[full_table["altitude"].notna()].reset_index(drop=True)
        drop_ind = []
        
        for ind in range(1,len(prior_table)):
            """
            everything is ok, observations step by step ↓
            """
            if prior_table['start'][ind] > prior_table['end'][ind-1]:
                continue
                """
                overlap in time ↓
                """
            else:
                """
                TIME SHIFT IF IND > 1
                """
                if ind > 1: #we calculate ind-2 
                    delta = prior_table['start'][ind-1] - prior_table['end'][ind-2]
                    slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                    if delta != 0*u.hour and delta < 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['end'][ind-2] + slew_time_i
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1] 
                    elif delta != 0*u.hour and delta > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour  + slew_time_i
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    else: #delta = 0h
                        prior_table['start'][ind] = prior_table['start'][ind] + slew_time_i
                        prior_table['end'][ind] = prior_table['end'][ind] +  prior_table['time_obs'][ind]

                    """
                    TIME SHIFT FOR 2D OBJECT (shift 1st object to sunset)
                    """
                else: #overlap for the 2d object ind = 1
                    delta_sunset = prior_table['start'][ind-1] - sunset_tonight
                    if delta_sunset != 0*u.hour and delta_sunset < 2*u.hour:
                        prior_table['start'][ind-1] = sunset_tonight
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    elif delta_sunset != 0*u.hour and delta_sunset > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    else: #delta = 0h
                        prior_table['start'][ind-1] = prior_table['start'][ind-1]
                        prior_table['end'][ind-1] = prior_table['start'][ind-1]

                    """
                    CHECK OVERLAP AGAIN
                    """        
                if prior_table['start'][ind] > prior_table['end'][ind-1]:
                    continue

                else:
                    overlap = prior_table['end'][ind-1] - prior_table['start'][ind]
                    if overlap < (prior_table['time_obs'][ind]/4): # 25% 
                        slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                        prior_table['start'][ind] = prior_table['start'][ind] + overlap  + slew_time_i
                        prior_table['end'][ind] = prior_table['end'][ind] + overlap  + slew_time_i
                    else:
                        m = int(prior_table['exp_num'][ind - 1])
                        while m > 3: # becuse we need as less 3 frames for each filter to make MasterFrame
                            #it relates with statistic issues 
                            m -= 1
                            if prior_table["binning"][ind-1]==1:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_1bin * 3)*u.second
                            elif prior_table["binning"][ind-1]==1:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_2bin * 3)*u.second
                            else:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_3bin * 3)*u.second

                            prior_table['end'][ind-1] = prior_table['end'][ind-1] - obs_time_1
                            overlap_m = prior_table['end'][ind-1] - prior_table['start'][ind]
                            #N - 1 worked
                            if overlap_m < (prior_table['time_obs'][ind]/4):#25%
                                slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                                prior_table['exp_num'][ind - 1] = m
                                prior_table['start'][ind] = prior_table['start'][ind] + overlap_m + slew_time_i
                                prior_table['end'][ind] = prior_table['end'][ind] + overlap_m + slew_time_i
                                break
                            #N - 1 didn't work. Repeat
                            else:
                                continue

                        # N < 3, we can't observe less than 3 series
                        # so we should take into accaunt priority
                        else:
                            """
                            Check percent of observation which were made
                            """
                            if float(prior_table['N_obs_perc'][ind-1]) > float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind)
                            elif float(prior_table['N_obs_perc'][ind-1]) < float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind-1)
                            else:
                                """
                                If percent of observation which were made is equal.
                                Check priority.
                                First off all, we want to give some observations for 
                                all type of objects (for all orders),
                                so we try to do observations homogeniously
                                But if the percent is equal - we have to look at priority.
                                """
                                if float(prior_table['priority'][ind-1]) > float(prior_table['priority'][ind]):
                                    drop_ind.append(ind)
                                elif float(prior_table['priority'][ind-1]) < float(prior_table['priority'][ind]):
                                    drop_ind.append(ind-1)
                                else:
                                    """
                                    If priority is equal and if we can calculate distances among 3 stars, we will choose 
                                    the nearest star, but if we cannot calculate distances we will choose 
                                    that star which culminate first.
                                    """
                                    if ind < 2:
                                        #if we cannot calculate distances among 3 stars, we will choose 
                                        #that star which culminate first
                                        if prior_table['start'][ind-1] > prior_table['start'][ind]: 
                                            drop_ind.append(ind)
                                        elif prior_table['start'][ind-1] < prior_table['start'][ind]:
                                            drop_ind.append(ind-1)
                                    else: # ind >= 2
                                        #if we can calculate distances among 3 stars, we will choose 
                                        #the nearest star
                                        distance_20 = ScheduleTable.calc_distance(self,prior_table['ra'][ind-2],prior_table['ra'][ind],prior_table['dec'][ind-2],prior_table['dec'][ind])
                                        distance_21 = ScheduleTable.calc_distance(self,prior_table['ra'][ind-2],prior_table['ra'][ind-1],prior_table['dec'][ind-2],prior_table['dec'][ind-1])
                                        if distance_20 > distance_21:
                                            drop_ind.append(ind)
                                        else:
                                            drop_ind.append(ind-1)
                
                """
                DROP objects, which will not observe this night
                """
        prior_table = prior_table.drop(labels = drop_ind).reset_index(drop=True)
    
        """
        Check if observations is duiring after sunrise
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time()#sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunrise_tonight = observatory_time[1]
        if prior_table['end'].iloc[len(prior_table)-1] > sunrise_tonight:
            slew_end_parking_overlap = ScheduleTable.calc_slew_time(self,prior_table['ra'].iloc[len(prior_table)-2], prior_table['ra'].iloc[len(prior_table)-2],prior_table['dec'].iloc[len(prior_table)-1], prior_table['dec'].iloc[len(prior_table)-2],self.Telescope.slew_rate) 
            prior_table['end'].iloc[len(prior_table)-1] = prior_table['end'].iloc[len(prior_table)-2] + slew_end_parking_overlap
            prior_table = prior_table.drop(labels=len(prior_table)-1, axis=0)
            if prior_table['end'].iloc[len(prior_table)-1] > sunrise_tonight:
                print("Observations are shifted too much")
            
            """
            Make new table with time without date.
            """
        prior_table_splitted = prior_table
        #for t in range(0,len(prior_table_splitted)):
        #    prior_table_splitted['start'][t] = prior_table_splitted['start'][t].iso.split(" ")[1]
        #    prior_table_splitted['end'][t] = prior_table_splitted['end'][t].iso.split(" ")[1]
        #    prior_table_splitted['culmination'][t] = prior_table_splitted['culmination'][t].iso.split(" ")[1]
        
        prior_table_splitted = prior_table_splitted.reset_index(drop=True)
        
        return prior_table_splitted
    
    def two_tables(self):

        
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time()#sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunrise_tonight = observatory_time[1]
        sunset_tonight = observatory_time[0]
        
        full_table =  ScheduleTable.change_priority_visib(self) # drop not observable objects and sort by ra
        prior_table_spl = ScheduleTable.check_overlap(self,table=full_table)
        #drop unnecessary columns

        
        prior_table_spl['N_obs_perc'] = round(prior_table_spl['exp_num']/prior_table_spl['N'],2)
        prior_table_spl['priority'] = round(prior_table_spl['priority'] - prior_table_spl['N_obs_perc'],2)
        
        for j in range(0,len(prior_table_spl)):
            for i in range(0,len(full_table)):
                if full_table['name'].iloc[i] == prior_table_spl['name'].iloc[j]:
                    full_table['priority'].iloc[i] = prior_table_spl['priority'].iloc[j]
                    full_table['N_obs_perc'].iloc[i] = prior_table_spl['N_obs_perc'].iloc[j]
                    
        #Find equatorial coordinates (RA,DEC) of telescope parking point.
        parking_alpha_deg_end, parking_dec_deg_end, parking_alpha_hms_end, parking_dec_dms_end = self.Observatory.parking_coordinates(date=sunrise_tonight)
        #parking_alpha_deg_start, parking_dec_deg_start, parking_alpha_hms_start, parking_dec_dms_start = self.Observatory.parking_coordinates(date=sunset_tonight)
        
        slew_parking_end=ScheduleTable.calc_slew_time(self,prior_table_spl["ra"].iloc[-1],parking_alpha_hms_end,prior_table_spl["dec"].iloc[-1],parking_dec_dms_end,self.Telescope.slew_rate)
        
        prior_table_spl = prior_table_spl.drop(["priority_vis","visible_time","visibility","priority","M","N_obs_perc","N"],axis=1)
        full_table = full_table.drop(["priority_vis","visible_time","visibility"],axis=1).reset_index(drop=True)
        
        # Parking point row. We should add it take into account slew time at the beginning and end of observation 
        obs_end = pd.DataFrame({'name':'parking_point', 'ra':parking_alpha_hms_end, 'dec':parking_dec_dms_end, 'B':np.NaN, 'V':np.NaN, 'R':np.NaN, 'binning':np.NaN, 'exp_num':np.NaN, 'start':np.NaN, 'end':prior_table_spl["end"].iloc[-1]+slew_parking_end, 'culmination':np.NaN, 'time_obs':np.NaN}, index =[0])
        
        prior_table_spl = pd.concat([prior_table_spl,obs_end]).reset_index(drop = True)
        
        for ind in range(0,len(prior_table_spl)-1):#
            prior_table_spl["start"][ind] =  prior_table_spl["start"][ind].iso.split(" ")[1]
            prior_table_spl["end"][ind] = prior_table_spl["end"][ind].iso.split(" ")[1]
            prior_table_spl["culmination"][ind] = prior_table_spl["culmination"][ind].iso.split(" ")[1]
            
        prior_table_spl["end"].iloc[-1] = prior_table_spl["end"].iloc[-1].iso.split(" ")[1]  

        return full_table, prior_table_spl

## Variable part

### Date

In [9]:
JD = Time('2023-03-01 00:00:00') #date of observations + 1 (because we need midnight)
#if we well make observations in night between 14th and 15th of February, we should write '2022-02-15 00:00:00'

In [ ]:
TELESCOPE =Telescope(readout_1bin,readout_2bin,readout_3bin,fil_turn_near,fil_turn_far,slew_rate)

TSHAO = Observatory(JD, "tshao", longitude_tshao, latitude_tshao)

### Stars to check functions

In [ ]:
regulus= Object("10 09 23.21","11 53 12", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

atria = Object("16 48 39.21","-69 53 12", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

kohlab = Object("16 48 39.21","74 09 00", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

aldhanab= Object("10 09 23.21","-37 21 57", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

sirius= Object("06 46 10.25","-16 45 03.2", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 16:12  ---  01.03 01:58

procyon= Object("07 39 17","5 12 59", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 15:41  ---  01.03 04:18

### Import data

In [43]:
data = pd.read_excel('objects_example_test.xlsx')

TELESCOPE =Telescope(readout_1bin,readout_2bin,readout_3bin,fil_turn_near,fil_turn_far,slew_rate)

TSHAO = Observatory(JD, "tshao", longitude_tshao, latitude_tshao)

In [84]:
st = time.time()

table = ScheduleTable(JD,data,TSHAO,TELESCOPE)

full_table, prior_table_spl = table.two_tables()

et = time.time()

print("Execute time is ", et-st, " seconds")

______________
WR 6 
2023-02-28 18:23:00.000
2023-02-28 12:03:00.000
______________
WR 9 
2023-02-28 18:56:00.000
2023-02-28 13:13:00.000
______________
WR 7a 
2023-02-28 18:50:00.000
2023-02-28 12:29:00.000
______________
WR 7 
2023-02-28 19:22:00.000
2023-02-28 11:53:00.000
______________
WR 10 
2023-02-28 19:18:00.000
2023-02-28 13:20:00.000
______________
WR 8 
2023-02-28 18:58:00.000
2023-02-28 13:10:00.000
______________
MWC560
2023-02-28 19:50:00.000
2023-02-28 11:40:00.000
______________
PN M1-6
2023-02-28 19:30:00.000
2023-02-28 10:19:00.000
______________
PN M1-9
2023-02-28 20:11:00.000
2023-02-28 10:37:00.000
______________
AX Per
2023-02-28 16:50:00.000
2023-03-01 02:58:00.000
______________
PN M1-5
2023-02-28 20:12:00.000
2023-02-28 08:00:00.000
______________
EXMPL 1
2023-02-28 23:22:00.000
2023-02-28 09:35:00.000
______________
EXMPL 2
2023-02-28 20:14:00.000
2023-02-28 13:53:00.000
______________
EXMPL 3
2023-02-28 20:58:00.000
2023-02-28 13:29:00.000
______________
EXM

/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["visible_time"][ind] = target.calc_obj_rise_set_15deg_time()
/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["visibility"][ind] = target.get_culm_low_altitude()
/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

Execute time is  21.357367992401123  seconds


/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_timing['start'][ind] = table_timing['start'][ind] - ScheduleTable.calc_slew_time(self,table_timing['ra'][ind-1],table_timing['ra'][ind],table_timing['dec'][ind-1],table_timing['dec'][ind],self.Telescope.slew_rate)
/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour
/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_47896/1421791566.p

In [58]:
full_table

,name,ra,dec,B,V,R,binning,exp_num,priority,M,N_obs_perc,N,altitude
0,AX Per,01 36 23,+54 15 02,40,30,10,1,5,1.96,24,0.04,120,22.808
1,WR 3,01 38 55.62,58 09 22.60,90,60,40,1,5,2.89,9,0.11,45,23.004
2,WR 4,02 41 11.67,56 43 49.80,60,40,30,2,7,2.83,6,0.17,42,33.519
3,WR 5,02 52 11.66,56 56 07.10,90,60,40,1,5,1.93,15,0.07,75,18.002
4,EXMPL 43,03 10 57.65,03 17 27.10,120,90,60,2,5,0.92,12,0.08,60,39.001
5,EXMPL 44,04 10 35.27,18 32 02.40,90,60,40,1,5,2.96,24,0.04,120,46.643
6,EXMPL 45,04 36 46.65,35 47 18,60,40,30,2,7,1.00,30,0.00,210,49.519
7,PN M1-5,05 46 50,+24 22 03,120,90,60,2,3,2.00,15,0.00,45,NaN
8,PN M1-6,06 35 45,-00 05 37,120,90,60,2,7,2.96,24,0.04,168,71.104
9,WR 6,06 54 13.04,-23 55 42,90,60,40,1,5,1.00,9,0.00,45,76.328


In [66]:
TSHAO.sunrise.iso

'2023-02-28 23:54:56.840'

In [85]:
prior_table_spl#.sort_values(by="ra")

,name,ra,dec,B,V,R,binning,exp_num,altitude,start,end,culmination,time_obs
0,AX Per,01 36 23,+54 15 02,40.0,30.0,10.0,1.0,5.0,22.808,15:59:06.739,16:11:36.739,18:05:21.739,750.0 s
1,WR 3,01 38 55.62,58 09 22.60,90.0,60.0,40.0,1.0,5.0,23.004,16:11:47.336,16:33:27.336,18:05:31.278,1300.0 s
2,WR 4,02 41 11.67,56 43 49.80,60.0,40.0,30.0,2.0,7.0,33.519,16:33:29.234,16:54:43.234,18:09:24.781,1274.0 s
3,WR 5,02 52 11.66,56 56 07.10,90.0,60.0,40.0,1.0,5.0,18.002,16:55:50.452,17:17:30.452,18:10:06.031,1300.0 s
4,EXMPL 43,03 10 57.65,03 17 27.10,120.0,90.0,60.0,2.0,5.0,39.001,17:17:56.825,17:44:46.825,18:11:16.405,1610.0 s
5,EXMPL 44,04 10 35.27,18 32 02.40,90.0,60.0,40.0,1.0,5.0,46.643,17:45:09.573,18:06:49.573,18:15:00.006,1300.0 s
6,PN M1-6,06 35 45,-00 05 37,120.0,90.0,60.0,2.0,7.0,71.104,18:17:30.516,18:33:44.516,18:24:04.364,2254.0 s
7,PN M1-9,07 05 19,+02 46 59,120.0,90.0,60.0,2.0,7.0,22.808,18:47:18.489,19:03:32.489,18:25:55.239,2254.0 s
8,MWC560,07 25 51,-07 44 09,40.0,30.0,20.0,1.0,5.0,36.180,19:26:27.794,19:34:07.794,18:27:12.239,800.0 s
9,EXMPL 3,08 54 59.16,-13 13 01.50,60.0,40.0,30.0,1.0,5.0,40.139,20:07:50.664,20:17:30.664,18:32:46.499,1000.0 s


In [59]:
prior_table_spl#.sort_values(by="ra")

,name,ra,dec,B,V,R,binning,exp_num,altitude,start,end,culmination,time_obs
0,AX Per,01 36 23,+54 15 02,40.0,30.0,10.0,1.0,5.0,22.808,15:59:06.739,16:11:36.739,18:05:21.739,750.0 s
1,WR 3,01 38 55.62,58 09 22.60,90.0,60.0,40.0,1.0,5.0,23.004,16:11:36.739,16:33:16.739,18:05:31.278,1300.0 s
2,WR 4,02 41 11.67,56 43 49.80,60.0,40.0,30.0,2.0,7.0,33.519,16:33:16.739,16:54:30.739,18:09:24.781,1274.0 s
3,WR 5,02 52 11.66,56 56 07.10,90.0,60.0,40.0,1.0,5.0,18.002,16:54:30.739,17:16:10.739,18:10:06.031,1300.0 s
4,EXMPL 43,03 10 57.65,03 17 27.10,120.0,90.0,60.0,2.0,5.0,39.001,17:16:10.739,17:43:00.739,18:11:16.405,1610.0 s
5,EXMPL 44,04 10 35.27,18 32 02.40,90.0,60.0,40.0,1.0,5.0,46.643,17:43:00.739,18:04:40.739,18:15:00.006,1300.0 s
6,PN M1-6,06 35 45,-00 05 37,120.0,90.0,60.0,2.0,7.0,71.104,18:13:54.739,18:30:08.739,18:24:04.364,2254.0 s
7,PN M1-9,07 05 19,+02 46 59,120.0,90.0,60.0,2.0,7.0,22.808,18:42:48.739,18:59:02.739,18:25:55.239,2254.0 s
8,MWC560,07 25 51,-07 44 09,40.0,30.0,20.0,1.0,5.0,36.180,19:20:56.739,19:28:36.739,18:27:12.239,800.0 s
9,EXMPL 3,08 54 59.16,-13 13 01.50,60.0,40.0,30.0,1.0,5.0,40.139,19:57:50.739,20:07:30.739,18:32:46.499,1000.0 s


In [ ]:
data['RA'] = ''*len(data)
data['dec'] = ''*len(data)
data['N'] = data['exp_num']*data['M']

for ind,ra in enumerate(data['ra']):
    dec = str(data['dec'][ind]).split(' ')
    ra_0 = str(ra).split(' ')
    ra_h = float(ra_0[0]) + float(ra_0[1])/60 + float(ra_0[2])/3600
    data['RA'][ind] = ra_h * 360 / 24 #recalculate RA from hours to degrees     
    data['dec'][ind] = float(dec[0]) + float(dec[1])/60 + float(dec[2])/3600
    
data_dec = data[data['dec'].apply(lambda x: x > -30)]
data_dec = data_dec.reset_index(drop = True)

data_dec = data_dec.sort_values('RA') #because RA connected with culmination time and in the future staeps we will need in sorted by culm time list

del data

### Calculations

In [ ]:
#Calculate sunset, sunrise in local and local sidereal time.
sunset_tonight, sunrise_tonight, sid_start, sid_end, time_range, sid_h_0 = sunset_rise_time(JD)

#Let's make two different lists with observable targets (FixedTarget) and their name
names,altitude = target_obs_list(sid_start, sid_end)

#make table with only observable objects
data_obs = data_dec[data_dec['name'].apply(lambda name: name in names)].reset_index(drop=True)
data_obs['Alt'] = altitude
#Find equatorial coordinates (RA,DEC) of telescope parking point.
parking_alpha_deg, parking_delta_deg, parking_alpha_hms, parking_dec_dms = parking_coordinates(phi,JD)

# Parking point row. We should add it take into account slew time at the beginning and end of observation 
new_row = pd.DataFrame({'name':'parking_point', 'ra':parking_alpha_deg, 'dec':parking_delta_deg, 'B':0, 'V':0, 'R':0, 'bin':1, 'n':0, 'priority':0, 'M':0, 'N_obs_perc': 0,'RA':parking_alpha_deg, 'dec':parking_delta_deg, 'N':0, 'Alt':0 }, index =[0])
df = pd.concat([new_row, data_obs]).reset_index(drop = True)

del data_dec
del data_obs

time_observ, obj_start, obj_end, culmination_time = time_calculation(df)

### Create a table with objects and their observing time 

In [ ]:
slew_end_parking = slew_time(df['RA'].iloc[0], df['dec'].iloc[0], df['RA'].iloc[-1], df['dec'].iloc[-1])    
obj_end.append((Time(obj_end[-1])+slew_end_parking))
obj_start.append('-')
culmination_time.append('-')
objects = np.append(np.array(df['name'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),'END')
ra = np.append(np.array(df['ra'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),parking_alpha_hms)
dec = np.append(np.array(df['dec'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),parking_dec_dms)
ra_deg = np.append(np.array(df['RA'][1:].apply(lambda x: float(str(x).replace(u'\xa0', u'')))),df['RA'].iloc[0])
dec_deg = np.append(np.array(df['dec'][1:].apply(lambda x: float(str(x).replace(u'\xa0', u'')))), df['dec'].iloc[0])
N = np.append(np.array(df['N'][1:]), 0)
B = np.append(np.array(df['B'][1:]), 0)
V = np.append(np.array(df['V'][1:]), 0)                        
R = np.append(np.array(df['R'][1:]), 0)                        
binning = np.append(np.array(df['bin'][1:]),0)
priority = np.append(np.array(df['priority'][1:]), 0)   
N_obs_perc = np.append(np.array(df['N_obs_perc'][1:]), 0) 
N_obs_perc = np.append(np.array(df['N_obs_perc'][1:]), 0) 
n = np.append(np.array(df['n'][1:]), 0)
M = np.append(np.array(df['M'][1:]), 0)
altitude = np.append(np.array(df['Alt'][1:].apply(lambda x: round(x.value,1))), 0)

time_observ.append(0*u.second)
              
table_schedule = pd.DataFrame({'name':objects, 'RA':ra, 'dec':dec, 'RA_deg':ra_deg, 'DEC_deg':dec_deg, 'B':B,'V':V,'R':R, 'bin':binning, 'N':N, 'priority':priority,'start':obj_start, 'culmination':culmination_time, 'end':obj_end, 'time_obs':time_observ, 'n':n, 'Alt':altitude, 'M':M, 'N_obs_perc':N_obs_perc})

del objects; del ra; del dec

### Overlap check

In [ ]:
prior_table_spl = overlap_check(table_schedule)

### Change priority of observed objects

In [ ]:
prior_table_spl['N_obs_perc'] = round(prior_table_spl['n']/prior_table_spl['N'],2)
prior_table_spl['priority'] = round(prior_table_spl['priority'] - prior_table_spl['N_obs_perc'],2)

### New table with all objects and their new priority

In [ ]:
table_new = read_ods('objects_example_test.ods')

In [ ]:
counter = 0
for j in range(0,len(prior_table_spl)):
    for i in range(0,len(table_new)):
        if table_new['name'].iloc[i].replace("\xa0","") == prior_table_spl['name'].iloc[j]:
            counter+=1
            table_new['priority'].iloc[i] = prior_table_spl['priority'].iloc[j]
            table_new['N_obs_perc'].iloc[i] = prior_table_spl['N_obs_perc'].iloc[j]

## tabulating in LaTeX format

In [ ]:
print(tabulate(prior_table_spl, headers=prior_table_spl.columns, tablefmt="latex"))

## Save tables

In [ ]:
#Save table with plan at night
d = JD.value.split(" ")[0]

prior_table_spl.to_excel(f'{d}_schedule.xlsx',index=False)
#Save table with all objects
table_new.to_excel(f'objs_new_prior.xlsx',index=False)

In [ ]:
longitude_tshao = '76d58m17.00s'
latitude_tshao = '43d03m29.00s'
elevation_tshao = 2735 * u.m
location_tshao = EarthLocation.from_geodetic(longitude_tshao, latitude_tshao, elevation_tshao)

tshao = Observer(name='tshao',
               location=location_tshao,
               pressure=0.615 * u.bar,
               relative_humidity=0.7,
               temperature=0 * u.deg_C,
               timezone=timezone('Asia/Almaty'),
               description="Tien Shan Astronomical Observatory, Kazaknstan")